In [1]:
import psycopg2
import requests
import pandas as pd
import json
from sqlalchemy import create_engine

In [2]:
conn = psycopg2.connect(
    database='riot',
    user='admin',
    password='admin',
    host='db',
    port='5432'
)

engine = create_engine('postgresql+psycopg2://', creator=lambda: conn)

In [3]:
cursor = conn.cursor()
cursor.execute("""SELECT table_name FROM information_schema.tables
       WHERE table_schema = 'public'""")
for table in cursor.fetchall():
    print(table)

('test',)
('dim_champions',)


In [4]:
champions_url = 'https://ddragon.leagueoflegends.com/cdn/13.22.1/data/en_US/champion.json'
res = requests.get(champions_url)
res_dict = json.loads(res.text)
champion_df_raw = pd.DataFrame(res_dict['data']).T
champion_df = champion_df_raw.set_index('key', drop=True)

In [5]:
champion_df.head()

,version,id,name,title,blurb,info,image,tags,partype,stats
key,,,,,,,,,,
266,13.22.1,Aatrox,Aatrox,the Darkin Blade,Once honored defenders of Shurima against the ...,"{'attack': 8, 'defense': 4, 'magic': 3, 'diffi...","{'full': 'Aatrox.png', 'sprite': 'champion0.pn...","[Fighter, Tank]",Blood Well,"{'hp': 650, 'hpperlevel': 114, 'mp': 0, 'mpper..."
103,13.22.1,Ahri,Ahri,the Nine-Tailed Fox,Innately connected to the magic of the spirit ...,"{'attack': 3, 'defense': 4, 'magic': 8, 'diffi...","{'full': 'Ahri.png', 'sprite': 'champion0.png'...","[Mage, Assassin]",Mana,"{'hp': 590, 'hpperlevel': 96, 'mp': 418, 'mppe..."
84,13.22.1,Akali,Akali,the Rogue Assassin,Abandoning the Kinkou Order and her title of t...,"{'attack': 5, 'defense': 3, 'magic': 8, 'diffi...","{'full': 'Akali.png', 'sprite': 'champion0.png...",[Assassin],Energy,"{'hp': 570, 'hpperlevel': 119, 'mp': 200, 'mpp..."
166,13.22.1,Akshan,Akshan,the Rogue Sentinel,"Raising an eyebrow in the face of danger, Aksh...","{'attack': 0, 'defense': 0, 'magic': 0, 'diffi...","{'full': 'Akshan.png', 'sprite': 'champion0.pn...","[Marksman, Assassin]",Mana,"{'hp': 630, 'hpperlevel': 107, 'mp': 350, 'mpp..."
12,13.22.1,Alistar,Alistar,the Minotaur,Always a mighty warrior with a fearsome reputa...,"{'attack': 6, 'defense': 9, 'magic': 5, 'diffi...","{'full': 'Alistar.png', 'sprite': 'champion0.p...","[Tank, Support]",Mana,"{'hp': 685, 'hpperlevel': 120, 'mp': 350, 'mpp..."


In [55]:
info_unpacked = champion_df.apply(lambda x: pd.Series(x['info']), axis=1, result_type='expand')
stats_unpacked = champion_df.apply(lambda x: pd.Series(x['stats']), axis=1, result_type='expand')
champion_df = champion_df \
                .join(info_unpacked) \
                .join(stats_unpacked)

champion_df.head()

,version,id,name,title,blurb,info,image,tags,partype,stats,...,hpregen,hpregenperlevel,mpregen,mpregenperlevel,crit,critperlevel,attackdamage,attackdamageperlevel,attackspeedperlevel,attackspeed
key,,,,,,,,,,,,,,,,,,,,,
266,13.22.1,Aatrox,Aatrox,the Darkin Blade,Once honored defenders of Shurima against the ...,"{'attack': 8, 'defense': 4, 'magic': 3, 'diffi...","{'full': 'Aatrox.png', 'sprite': 'champion0.pn...","[Fighter, Tank]",Blood Well,"{'hp': 650, 'hpperlevel': 114, 'mp': 0, 'mpper...",...,3.00,1.00,0.0,0.0,0.0,0.0,60.0,5.00,2.500,0.651
103,13.22.1,Ahri,Ahri,the Nine-Tailed Fox,Innately connected to the magic of the spirit ...,"{'attack': 3, 'defense': 4, 'magic': 8, 'diffi...","{'full': 'Ahri.png', 'sprite': 'champion0.png'...","[Mage, Assassin]",Mana,"{'hp': 590, 'hpperlevel': 96, 'mp': 418, 'mppe...",...,2.50,0.60,8.0,0.8,0.0,0.0,53.0,3.00,2.200,0.668
84,13.22.1,Akali,Akali,the Rogue Assassin,Abandoning the Kinkou Order and her title of t...,"{'attack': 5, 'defense': 3, 'magic': 8, 'diffi...","{'full': 'Akali.png', 'sprite': 'champion0.png...",[Assassin],Energy,"{'hp': 570, 'hpperlevel': 119, 'mp': 200, 'mpp...",...,9.00,0.90,50.0,0.0,0.0,0.0,62.0,3.30,3.200,0.625
166,13.22.1,Akshan,Akshan,the Rogue Sentinel,"Raising an eyebrow in the face of danger, Aksh...","{'attack': 0, 'defense': 0, 'magic': 0, 'diffi...","{'full': 'Akshan.png', 'sprite': 'champion0.pn...","[Marksman, Assassin]",Mana,"{'hp': 630, 'hpperlevel': 107, 'mp': 350, 'mpp...",...,3.75,0.65,8.2,0.7,0.0,0.0,52.0,3.00,4.000,0.638
12,13.22.1,Alistar,Alistar,the Minotaur,Always a mighty warrior with a fearsome reputa...,"{'attack': 6, 'defense': 9, 'magic': 5, 'diffi...","{'full': 'Alistar.png', 'sprite': 'champion0.p...","[Tank, Support]",Mana,"{'hp': 685, 'hpperlevel': 120, 'mp': 350, 'mpp...",...,8.50,0.85,8.5,0.8,0.0,0.0,62.0,3.75,2.125,0.625


In [67]:
champion_df.drop(
        columns=['info', 'stats', 'image']
    ) \
    .to_sql(
        name='dim_champions',
        con=engine,
        if_exists='replace',
        index=True,
        index_label='champion_id',
        method='multi'
    )

165